<a href="https://colab.research.google.com/github/Deep-Dhaduk/AutoGluon/blob/main/Fraud-detection/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 200)

In [5]:
!pip install --quiet autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.9/454.9 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.7 MB/s eta 0:0

In [7]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

In [8]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [9]:
train_data['isFraud'].value_counts()

,count
isFraud,
0,569877
1,20663


In [10]:
from sklearn.model_selection import train_test_split
train_data, temp_data = train_test_split(train_data, test_size=0.2, random_state=42,stratify=train_data['isFraud'])
test_data, val_data = train_test_split(temp_data, test_size=0.5, random_state=42,stratify=temp_data['isFraud'])

In [11]:
train_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,...,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
40809,3027809,0,1008491,100.00,R,6177,399.0,150.0,american express,150.0,credit,264.0,87.0,NaN,1.0,anonymous.com,anonymous.com,1.0,1.0,0.0,2.0,0.0,1.0,0.0,2.0,0.0,2.0,1.0,0.0,2.0,1.0,609.0,609.0,NaN,NaN,NaN,NaN,NaN,609.666687,0.666666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,58410.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,-360.0,Found,Found,166.0,NaN,300.0,214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Windows 7,ie 11.0 for desktop,24.0,1920x1080,match_status:2,T,F,T,T,desktop,Trident/7.0
285886,3272886,0,7008212,29.99,W,7900,345.0,150.0,mastercard,224.0,debit,143.0,87.0,4.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,T,T,T,M0,T,F,F,F,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104256,3091256,0,2071522,107.95,W,11690,111.0,150.0,visa,226.0,credit,191.0,87.0,NaN,NaN,comcast.net,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,15.0,1.0,501.0,501.0,18.0,502.0,18.0,NaN,NaN,NaN,NaN,502.0,NaN,NaN,NaN,NaN,502.0,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,200.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
507860,3494860,0,13299752,241.95,W,2616,327.0,150.0,discover,102.0,credit,330.0,87.0,3.0,NaN,NaN,NaN,1.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,1.0,177.0,177.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,177.0,177.0,NaN,NaN,NaN,177.0,T,T,F,NaN,NaN,T,F,F,T,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,

In [12]:
train_data.shape

(472432, 434)

In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 472432 entries, 40809 to 337290
Columns: 434 entries, TransactionID to DeviceInfo
dtypes: float64(399), int64(4), object(31)
memory usage: 1.5+ GB


In [14]:
train_data.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,...,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_13,id_14,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
count,4.724320e+05,472432.000000,4.724320e+05,472432.000000,472432.000000,465285.000000,471197.000000,469035.00000,419709.000000,419709.000000,190428.000000,30156.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,472432.000000,471398.000000,247635.000000,261839.000000,337134.000000,224193.000000,58756.000000,31179.000000,60129.000000,60129.000000,411587.000000,248762.000000,51953.000000,49740.000000,49944.000000,401062.000000,248762.000000,248762.000000,248762.000000,248762.000000,248762.000000,248762.000000,248762.000000,248762.000000,248762.000000,248762.000000,248762.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,411548.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,337096.000000,410643.000000,410643.000000,410643.000000,410643.000000,410643.000000,410643.000000,410643.000000,...,104634.000000,104634.000000,104634.000000,104634.000000,104634.000000,104634.000000,104634.000000,113426.000000,113426.000000,113426.000000,104634.000000,104634.000000,104634.000000,104634.000000,104634.000000,104634.000000,472424.000000,472424.000000,471398.000000,471398.000000,471398.000000,472424.000000,472424.000000,472424.000000,472424.000000,471398.000000,471398.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,471398.000000,472424.000000,472424.000000,472424.000000,471398.000000,471398.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,471398.000000,471398.000000,471398.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,472424.000000,65995.000000,65995.000000,65995.000000,65995.000000,65995.000000,65995.000000,65995.00000,65995.000000,65995.000000,65995.000000,65995.000000,65995.000000,65995.000000,65995.000000,65995.000000,65995.000000,65995.000000,65995.000000,115658.000000,112989.000000,53192.000000,53192.000000,109791.000000,109791.000000,4114.000000,4114.000000,60129.000000,60129.000000,113079.000000,102085.000000,64168.000000,111801.000000,36124.000000,111759.000000,111713.000000,4119.000000,4126.000000,3793.000000,4102.000000,4122.000000,62203.000000
mean,3.282316e+06,0.034989,7.373394e+06,135.071756,9903.027720,362.527137,153.203514,199.30153,290.762967,86.799525,118.583764,231.967933,14.032987,15.208627,0.005542,4.070306,5.552901,9.037030,2.822131,5.110992,4.467680,5.204834,10.198232,4.040867,32.418708,8.266002,94.290778,169.547641,

In [15]:
train_data.columns.to_list()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V

In [16]:
(train_data.isna().sum()/train_data.shape[0]).sort_values(ascending=False)

,0
id_24,0.991971
id_25,0.991317
id_07,0.991292
id_08,0.991292
id_21,0.991281
...,...
ProductCD,0.000000
TransactionAmt,0.000000
TransactionDT,0.000000
isFraud,0.000000


In [17]:
train_data['id_24'].isna().sum()

np.int64(468639)

In [18]:
train_data = train_data[:50000]

In [19]:
train_data.shape

(50000, 434)

In [20]:
from datetime import time
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='medium_quality',tuning_data=val_data, time_limit=600, num_gpus=1
)

results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          8
GPU Count:          1
Memory Avail:       43.63 GB / 50.99 GB (85.6%)
Disk Space Avail:   179.49 GB / 235.68 GB (76.2%)
Presets specified: ['medium_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': False}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.1

[50]	valid_set's binary_logloss: 0.100628
[100]	valid_set's binary_logloss: 0.094533
[150]	valid_set's binary_logloss: 0.0912681
[200]	valid_set's binary_logloss: 0.089794
[250]	valid_set's binary_logloss: 0.0888166
[300]	valid_set's binary_logloss: 0.0882118
[350]	valid_set's binary_logloss: 0.087822
[400]	valid_set's binary_logloss: 0.0875146
[450]	valid_set's binary_logloss: 0.0874778
[500]	valid_set's binary_logloss: 0.0873865
[550]	valid_set's binary_logloss: 0.087331
[600]	valid_set's binary_logloss: 0.0874516
[650]	valid_set's binary_logloss: 0.087537
[700]	valid_set's binary_logloss: 0.0877828


Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/LightGBMXT/model.pkl
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/utils/attr/LightGBMXT/y_pred_proba_val.pkl
	0.9015	 = Validation score   (roc_auc)
	19.4s	 = Training   runtime
	1.79s	 = Validation runtime
	32958.1	 = Inference  throughput (rows/s | 59054 batch size)
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/trainer.pkl
Fitting model: LightGBM ... Training model for up to 569.88s of the 569.87s of remaining time.
	Fitting LightGBM with 'num_gpus': 1, 'num_cpus': 4
	Fitting with cpus=4, gpus=1, mem=0.9/43.0 GB
	Training LightGBM with GPU, note that this may negatively impact model quality compared to CPU training.
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'device': 'gpu'}


[50]	valid_set's binary_logloss: 0.0966024
[100]	valid_set's binary_logloss: 0.0912058
[150]	valid_set's binary_logloss: 0.0888264
[200]	valid_set's binary_logloss: 0.0877186
[250]	valid_set's binary_logloss: 0.0867219
[300]	valid_set's binary_logloss: 0.0864514
[350]	valid_set's binary_logloss: 0.0862687
[400]	valid_set's binary_logloss: 0.0863432
[450]	valid_set's binary_logloss: 0.086572


Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/LightGBM/model.pkl
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/utils/attr/LightGBM/y_pred_proba_val.pkl
	0.9069	 = Validation score   (roc_auc)
	8.42s	 = Training   runtime
	0.9s	 = Validation runtime
	65738.8	 = Inference  throughput (rows/s | 59054 batch size)
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/trainer.pkl
Fitting model: RandomForestGini ... Training model for up to 560.51s of the 560.50s of remaining time.
	Fitting RandomForestGini with 'num_gpus': 1, 'num_cpus': 8
	Fitting with cpus=8, gpus=1, mem=0.0/43.0 GB
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/RandomForestGini/model.pkl
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/utils/attr/RandomForestGini/y_pred_proba_val.pkl
	0.8802	 = Validation score   (ro

...

[0]	validation_0-logloss:0.13363
[50]	validation_0-logloss:0.09315
[100]	validation_0-logloss:0.09009
[150]	validation_0-logloss:0.08843
[200]	validation_0-logloss:0.08729
[250]	validation_0-logloss:0.08703
[300]	validation_0-logloss:0.08702
[350]	validation_0-logloss:0.08693
[394]	validation_0-logloss:0.08715


/usr/local/lib/python3.12/dist-packages/xgboost/core.py:729: UserWarning: [01:02:43] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/XGBoost/model.pkl
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/utils/attr/XGBoost/y_pred_proba_val.pkl
	0.9026	 = Validation score   (roc_auc)
	6.61s	 = Training   runtime
	1.8s	 = Validation runtime
	32895.2	 = Inference  throughput (rows/s | 59054 batch size)
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud

[50]	valid_set's binary_logloss: 0.0986407
[100]	valid_set's binary_logloss: 0.0913516
[150]	valid_set's binary_logloss: 0.0891788
[200]	valid_set's binary_logloss: 0.089345


Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/LightGBMLarge/model.pkl
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/utils/attr/LightGBMLarge/y_pred_proba_val.pkl
	0.9055	 = Validation score   (roc_auc)
	14.97s	 = Training   runtime
	0.96s	 = Validation runtime
	61198.4	 = Inference  throughput (rows/s | 59054 batch size)
Saving /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/trainer.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/utils/attr/LightGBMXT/y_pred_proba_val.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/utils/attr/RandomForestEntr/y_pred_proba_val.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/utils/attr/RandomForestGini/y_pred_proba_val.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluo

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.913558     roc_auc       5.765288  147.908514                0.009449           2.372238            2       True         12
1              LightGBM   0.906887     roc_auc       0.898313    8.423219                0.898313           8.423219            1       True          2
2         LightGBMLarge   0.905451     roc_auc       0.964960   14.969553                0.964960          14.969553            1       True         11
3               XGBoost   0.902599     roc_auc       1.795214    6.608451                1.795214           6.608451            1       True          9
4            LightGBMXT   0.901515     roc_auc       1.791793   19.399046                1.791793          19.399046            1       True          1
5              CatBoost   

In [21]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.913558,roc_auc,5.765288,147.908514,0.009449,2.372238,2,True,12
1,LightGBM,0.906887,roc_auc,0.898313,8.423219,0.898313,8.423219,1,True,2
2,LightGBMLarge,0.905451,roc_auc,0.964960,14.969553,0.964960,14.969553,1,True,11
3,XGBoost,0.902599,roc_auc,1.795214,6.608451,1.795214,6.608451,1,True,9
4,LightGBMXT,0.901515,roc_auc,1.791793,19.399046,1.791793,19.399046,1,True,1
5,CatBoost,0.899965,roc_auc,0.796548,104.696479,0.796548,104.696479,1,True,5
6,RandomForestEntr,0.883320,roc_auc,1.163429,11.579385,1.163429,11.579385,1,True,4
7,RandomForestGini,0.880236,roc_auc,1.390194,15.376461,1.390194,15.376461,1,True,3
8,ExtraTreesEntr,0.879973,roc_auc,1.300804,10.838575,1.300804,10.838575,1,True,7
9,ExtraTreesGini,0.877093,roc_auc,1.369949,11.891398,1.369949,11.891398,1,True,6


In [22]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
y_true = test_data[label].astype(int)
y_pred = predictor.predict(test_data.drop(columns=[label]),model='LightGBM')
AUC = roc_auc_score(y_true, y_pred)
print(AUC)

Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/LightGBM/model.pkl


0.6999806413211191


In [23]:
y_pred.value_counts(),y_true.value_counts()

(isFraud
 0    58105
 1      949
 Name: count, dtype: int64,
 isFraud
 0    56987
 1     2067
 Name: count, dtype: int64)

In [24]:
predictor.feature_importance(test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['V113', 'V119', 'V147', 'V149', 'V154', 'V156', 'V198', 'V241']
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl
Computing feature importance via permutation shuffling for 425 features using 5000 rows with 5 shuffle sets...
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/CatBoost/model.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/ExtraTreesEntr/model.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/LightGBM/model.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/LightGBMLarge/model.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/XGBoost/model.pkl
Loading: /content/drive/MyDrive/

,importance,stddev,p_value,n,p99_high,p99_low
C13,0.008521,0.001359,0.000075,5,0.011318,0.005724
C1,0.008518,0.002146,0.000446,5,0.012937,0.004098
TransactionAmt,0.007212,0.004616,0.012523,5,0.016717,-0.002293
card6,0.006791,0.002619,0.002199,5,0.012183,0.001399
C11,0.006419,0.002103,0.001205,5,0.010749,0.002089
...,...,...,...,...,...,...
V52,-0.000390,0.000660,0.871831,5,0.000968,-0.001749
V51,-0.000462,0.000435,0.961643,5,0.000435,-0.001358
D5,-0.000482,0.000432,0.966479,5,0.000407,-0.001372
V258,-0.000498,0.001128,0.810192,5,0.001825,-0.002820


In [25]:
predictor.positive_class

1

In [26]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

[0, 1]

In [27]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/CatBoost/model.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/ExtraTreesEntr/model.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/LightGBM/model.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/LightGBMLarge/model.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/XGBoost/model.pkl
Loading: /content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl


In [28]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)